# Calcium

在 `braincell` 中，我们通过对 `Ion` 类的扩展，来建模各种具体类型的离子。我们现在聚焦于钙离子 `Calcium` 的建模方式。

钙离子在神经元中起着至关重要的作用，参与动作电位的触发、神经递质的释放以及各种钙依赖的信号转导路径。因此，针对不同建模需求，我们提供了多个版本的钙离子模型：

-`CalciumFixed`：固定浓度与反转电位的钙离子模型
-`CalciumDetailed`：基于生物物理机制的精细钙动力学模型
-`CalciumFirstOrder`：简化的一阶钙动力学模型

当然，在对这些不同模型进行学习之前，我们先要对基类 `Calcium` 进行说明。

`Calcim` 是所有钙离子模型的基类。它本身不包含任何具体动力学，仅作为语义上的标识使用，即 `Ion` 的钙离子特化版本，方便对钙离子进行统一管理与约束。

## `CalciumFixed`

`CalciumFixed` 提供最简单的钙建模形式，假设钙离子的反转电位 `E` 与浓度 `C` 是固定值，没有随时间变化的动力学。

## `CalciumDetailed`

### 细胞内钙离子的动力学

细胞内钙离子的变化主要由两个机制决定：

#### 由钙电流引起的内流

钙离子通过钙通道进入细胞，并扩散到细胞内部。模型中只考虑膜下薄壳区域的钙离子浓度。内流的公式为：

$$
[Ca]_i = -\frac{I_{Ca}}{2 F d}
$$

其中：
- \$F = 96489, \mathrm{C/mol}\$ 为法拉第常数
- \$d = 1, \mu\mathrm{m}\$ 为膜下薄壳区域的厚度
- \$I\_{Ca}\$ 为所有钙通道产生的总电流
- $\[Ca]\_i\$ 为细胞内钙浓度

#### 由钙泵引起的外流

在膜下区域，钙的移除可通过结合缓冲物、钙泵活动、扩散等多种方式进行。此处模型只对泵进行建模，并采用如下动力学反应过程：

$$
\mathrm{Ca^{2+}_i} + P \underset{c_2}{\overset{c_1}{\rightleftharpoons}} \mathrm{CaP} \xrightarrow{c_3} P + \mathrm{Ca^{2+}_0}
$$

其中：
- \$P\$ 表示钙泵
- \$\mathrm{CaP}\$ 是钙泵中间复合态
- \$\mathrm{Ca^{2+}\_0}\$ 是膜外钙浓度
- \$c\_1\$, \$c\_2\$, \$c\_3\$ 为反应速率常数

考虑 \$c\_3\$ 很小，可使用 Michaelis-Menten 近似公式表示钙泵的动力学行为：

$$
\frac{d[Ca^{2+}]_i}{dt} = -\frac{K_T [Ca]_i}{[Ca]_i + K_d}
$$

其中：
- \$K\_T = 10^{-4}, \mathrm{mM/ms}\$
- \$K\_d = \frac{c\_2}{c\_1} = 10^{-4}, \mathrm{mM}\$
- \$K\_d\$ 表示钙泵激活一半所需的钙浓度

### 简化的一阶模型

在 Bazhenov（1998）提出的模型中，钙离子动力学可以简化为一阶线性微分方程：

$$
\frac{d[Ca^{2+}]_i}{dt} = -\frac{I_{Ca}}{z F d} + \frac{[Ca^{2+}]_{\text{rest}} - [Ca^{2+}]_i}{\tau_{Ca}}
$$

其中：
- \$I\_{Ca}\$：所有钙通道产生的总电流
- \$d\$：膜下薄壳的厚度（\$1, \mu\mathrm{m}\$）
- \$z = 2\$：\$\mathrm{Ca^{2+}}\$ 的电荷价
- \$F\$：法拉第常数
- \$\tau\_{Ca}\$：钙的清除时间常数
- $\[Ca^{2+}]\_{\text{rest}} = 0.05\mu\mathrm{M}\$


### 钙离子的反转电位

钙离子的反转电位根据 Nernst 方程给出：

$$
E = \frac{RT}{2F} \log \left( \frac{[Ca^{2+}]_0}{[Ca^{2+}]_i} \right)
$$

其中：
- \$R = 8.31441, \mathrm{J/(mol\cdot K)}\$：气体常数
- \$T = 309.15, \mathrm{K}\$：绝对温度（约为 36℃）
- \$F = 96489, \mathrm{C/mol}\$：法拉第常数
- $\[Ca^{2+}]\_0 = 2, \mathrm{mM}\$：膜外钙浓度

## `CalciumFirstOrder`

该模型采用一个简洁的一阶微分方程模拟钙浓度的变化：

$$
 Ca' = -\alpha I_{Ca} + -\beta Ca
 $$

至此，我们认识了 `Calcium` 中内置的离子模型。